In [1]:
import pandas as pd
import numpy as np
from Icarusight.queries.get_queries import query_product_df, query_co_viewed_df
from Icarusight.Icarusight.utils import *
from Icarusight.Icarusight.flatXcoviewed_model.flatXcoviewed_model import create_flat_x_coviewed_graph
from Icarusight.Icarusight.clustering import get_cluster_labels, apply_clustering_algorithm
from Icarusight.Icarusight.vectorizer.custom_tfidf_transformer_class import ClassTfidfTransformer
from collections import Counter
from nltk.corpus import stopwords
import nltk
from cdiscount import snowflake, config
import os
from numpy.linalg import norm
from scipy import sparse

ImportError: cannot import name 'query_product_df' from 'Icarusight.queries.get_queries' (/home/jupyter/Icarusight/queries/get_queries.py)

In [3]:
os.chdir("Icarusight")

In [6]:
secrets = config.load_secrets('queries/secrets.yml', key='snowflake')
con = snowflake.get_snowflake_connection(**secrets)
print("Starting queries.", end="")
product_df = query_product_df(con)
print(".", end="")
co_viewed_df = query_co_viewed_df(con)
print(" Done, length of co_viewed is: ", len(co_viewed_df))

print("Starting graph creation")
g = create_flat_x_coviewed_graph(product_df, co_viewed_df)
print("Graph creation done.", end=" ")
print(f"got {len(g.nodes)} nodes and {len(g.edges)} edges.")

print("Starting clustering...")
partitions = apply_clustering_algorithm(g)
print("Clustering done, removing the meaningless clusters...")
partitions = remove_meaningless_clusters(partitions, n=3)
print("Removing done.")

Starting queries.. Done, length of co_viewed is:  886462
Starting graph creation
Adding nodes...


100%|██████████| 95903/95903 [01:29<00:00, 1072.58it/s]


Nodes added, adding edges...


100%|██████████| 886462/886462 [00:10<00:00, 88529.76it/s]


Edges added.
Graph creation done. got 95903 nodes and 443231 edges.
Starting clustering...
Clustering done, removing the meaningless clusters...
Removing done.


In [20]:
print(product_df.columns)

Index(['product_name', 'product_category_name', 'product_category_code_path',
       'seller_names', 'brand_name', 'product_long_description', 'cluster_id'],
      dtype='object')


In [23]:
# Group by 'ref_category' and 'related_category' and count occurrences
pair_counts = co_viewed_df.groupby(['ref_category_name', 'related_category_name']).size().reset_index(name='count')
partial_product_df = product_df.where(product_df['cluster_id'].isin(partitions.values()))
categories_count = partial_product_df.groupby(['cluster_id', 'product_category_name']).size().reset_index(name='count')
categories_count = categories_count.where(categories_count['count'] > 10)
# Display the counts
print(categories_count.dropna())

    cluster_id            product_category_name   count
0          1.0            CANAPE - SOFA - DIVAN  2703.0
2         68.0            CANAPE - SOFA - DIVAN  1033.0
4         87.0            CANAPE - SOFA - DIVAN  1603.0
5        144.0                        BANQUETTE   241.0
7        162.0            CANAPE - SOFA - DIVAN  1833.0
9        184.0                 ENSEMBLE CANAPES   341.0
11       226.0  CLIC-CLAC - BANQUETTE CLIC-CLAC   624.0
12       715.0            CANAPE - SOFA - DIVAN    62.0
13      1058.0                BZ - BANQUETTE BZ    70.0
14      1357.0            CANAPE - SOFA - DIVAN  1807.0
15      2264.0                       MERIDIENNE   243.0
20      5470.0            CANAPE - SOFA - DIVAN    37.0
30     14047.0                       MERIDIENNE    29.0


In [5]:
def save_results(path_to_save, cluster_infos, cluster_top_words_count, cluster_top_words_tfidf):
    # Create a DataFrame from the dictionaries
    cluster_labels, cluster_max_values, cluster_total_len = cluster_infos[0], cluster_infos[1], cluster_infos[2]
    df = pd.DataFrame({
        'cluster_id': list(cluster_labels.keys()),
        'label': list(cluster_labels.values()),
        'max_values': list(cluster_max_values.values()),
        'total_len': list(cluster_total_len.values()),
        'top_words_count_method': list(cluster_top_words_count),
        'top_words_tfidf_method': list(cluster_top_words_tfidf)
    })

    # Write the DataFrame to a CSV file
    df.to_csv(path_to_save, index=False)


def remove_meaningless_clusters(input_dict, n):
    """
    Reduce the number of clusters by removing the meaningless ones.
    @param input_dict: Corresponds to the output partitions from the clustering algorithm under the form {node_id:cluster_id}
    @type input_dict: dict[str:str]
    @param n: Filter the clusters on their number of nodes in it. Keeps only the clusters with more than n nodes.
    @type n: int
    @return: Returns the input_dict without the meaningless nodes (i.e: clusters)
    @rtype: dict[str:str]
    """
    df = pd.DataFrame({"node_id": input_dict.keys(), "cluster_id": input_dict.values()})
    count_df = df.groupby("cluster_id", as_index=False).size()
    count_df = count_df.loc[lambda x: x["size"] > n]  # Filtering by size > n
    df = df.merge(right=count_df.drop(columns=["size"]), how="inner", on="cluster_id")
    return {node_id: cluster_id for node_id, cluster_id in df.itertuples(index=False)}

In [30]:
#product_df.set_index("product_id", inplace=True)

product_df['cluster_id'] = partitions
product_restrained_df = product_df[~product_df['cluster_id'].isnull()]
product_restrained_df.astype({'cluster_id': int})
product_restrained_df['name_descr'] = product_restrained_df.product_name +  ". " + product_restrained_df.product_long_description + ". "
cluster_serie = product_restrained_df.groupby("cluster_id").name_descr.sum()
print(cluster_serie)

/tmp/ipykernel_355782/1920937987.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_restrained_df['name_descr'] = product_restrained_df.product_name +  ". " + product_restrained_df.product_long_description + ". "


cluster_id
1.0        ZHI Canapé lit à 2 places Noir Tissu 738650263...
68.0       Canapé lit élégant Canapé moderne Intérieur mo...
87.0       Canapé convertible 3 places avec coffre de ran...
144.0      MOO Canapé lit à 2 places Vert foncé Velours 7...
162.0      Canapé lit à 2 places et deux oreillers Rouge ...
184.0      Canapé 3 places et fauteuil CHESTERFIELD Simil...
226.0      Canapé lit STAR scandinave Sofa convertible av...
715.0      Canapé Chesterfield en forme de L Cuir synthét...
1058.0     Banquette lit BZ matelas HR 140 cm YASMO n 2. ...
1357.0     Canapé d angle panoramique Dante en U en velou...
2264.0     Beliani Chaise longue rose poudré côté droit M...
2377.0     Canapé lit à 2 places Gris clair Velours DIOCH...
4303.0     Micadoni Home JUSTIN Canapé d angle 4 places e...
5138.0     Canapé lit à 2 places Marron Microfibre SALALI...
5407.0     YaJiaSheng Ensemble de canapés à 2 et à 3 plac...
5470.0     LVL MEUBLE SOFA Canapé lit à 2 places Rose Vel...
5505.0     AB

In [31]:
def get_top_tf_idf_words(response, threshold, top_n=2):
    #response_normalized = normalize(response, axis=1, norm='l2')
    response_normalized = custom_norm(response)
    response_normalized.data[response_normalized.data < threshold] = 0.0
    response_normalized.eliminate_zeros()
    #print(response_normalized.data)
    sorted_nzs = np.argsort(response_normalized.data)[:-(top_n+1):-1]
    res = feature_array[response_normalized.indices[sorted_nzs[ response_normalized.indices[sorted_nzs] > threshold]]]
    return np.apply_along_axis(' | '.join, 0, res)

def custom_norm(x):
    norm = x.sum(axis=1)
    return x / norm

from sklearn.preprocessing import normalize

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
 
# COUNT VECTORIZER METHOD
vectorizer = CountVectorizer(stop_words=nltk.corpus.stopwords.words('french'))
vector = vectorizer.fit_transform(cluster_serie)
top_words_per_cluster_count = [get_top_tf_idf_words(item, 0.015, 5) for item in vector]
for top_words in top_words_per_cluster_count:
    print(top_words)

cm | canapé | lit | siège | hauteur
canapé | angle | cm | convertible | places
canapé | cm | convertible | places | express
banquette | cm | velours | pieds | tissu
canapé | places | cm | cuir
places | canapé | cm | ensemble | siège
cm | canapé | lit | hauteur | dimensions
cm | siège | cuir | hauteur | canapé
cm | bz | banquette | matelas | couchage
canapé | angle | convertible | tissu | places
méridienne | cm | pieds | kg | velours
cm | hauteur | siège | canapé | lit
cm | poids | kg | total | meridienne
cm | hauteur | siège | canapé | marron
rotin | cm | canapé | coussin | couleur
cm | hauteur | lit | canapé | siège
cm | siège | hauteur | microfibre | lit
canapé | angle | convertible | alma | cm
poids | kg | volume | net | pieds
cm | poids | kg | meridienne | total
cm | poids | kg | accoudoir | hauteur
cm | hauteur | siège | microfibre | canapé
cm | poids | kg | total | largeur
cm | canapé | siège | hauteur | places
cm | siège | rouge | canapé | tissu
cm | poids | kg | net | meridienn

In [33]:
class CustomClassTfidfTransformer(ClassTfidfTransformer):
    def __init__(self, use_idf: bool = False, bm25_weighting: bool = False, reduce_frequent_words: bool = False):
        super(CustomClassTfidfTransformer, self).__init__(
            bm25_weighting=bm25_weighting,
            reduce_frequent_words=reduce_frequent_words
        )
        self.use_idf = use_idf
        
    def transform(self, X):
        X = normalize(X, axis=1, norm='l1', copy=False)

        if self.use_idf:

            if self.reduce_frequent_words:
                X.data = np.sqrt(X.data)

            X = X * self._idf_diag

        return X

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TFIDF VECTORIZER METHOD
v = CustomClassTfidfTransformer(use_idf=True)
x = v.fit_transform(vector)
feature_array = np.array(vectorizer.get_feature_names_out())
tfidf_sorting = np.argsort(x.toarray()).flatten()[::-1]
top_words_per_cluster_tfidf = [get_top_tf_idf_words(item, 0.015, 5) for item in x]
for top_words in top_words_per_cluster_tfidf:
    print(top_words)


angle
express
banquette

ensemble | canapés | fauteuil
clic | clac
synthétique | 73 | 94 | siège | cuir
bz | banquette | france
angle
méridienne | meridienne
hauteur | clair | 69 | ab337299 | siège
meridienne | 71872 | 829744 | 236x160x72 | poids
microfibre | marron | 220 | doss | 69
rotin | véritable | naturel | véranda | canapés
rose | hauteur | 69 | épaisseur | siège
microfibre | marron | vinyle | polychlorure | 69
alma | 254 | 141 | 87 | 150
220x92x90 | 44775 | 8216 | poids | kg
meridienne | 284x166x70 | 697552 | 30008 | poids
meridienne | poids | kg | total | volume
microfibre | crème | ako7394667535542 | polychlorure | vinyle
10256 | 252x220x74 | 550 | 738736 | meridienne
fixe | siège | assemblage | partir | sol
rouge | bordeaux | 117167 | a348967 | estink
meridienne | poids | kg | total | volume
278x220x74 | sovite | structurel | 100000 | constitué
soulagent | continu | imposante | 270x125 | merveilleusement
liy3080546 | 262 | dimensions | 35 | pied
70 | 73 | siege | pcs | cous

In [37]:
cluster_infos = get_cluster_labels(g, partitions)
print("Saving results...")
save_results("results/results_threshold_at_0.015_cutoff_2.csv", cluster_infos, top_words_per_cluster_count, top_words_per_cluster_tfidf)
print(os.getcwd())
print("Results saved.")

100%|██████████| 31/31 [00:00<00:00, 302380.06it/s]

Saving results...
/home/jupyter/Icarusight
Results saved.
